**Langchain Tutorial**

LangChain is a framework for developing applications powered by language models. It enables applications that


**Are context-aware:** connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)

**Reason:** rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)

This following code contains

1. Call API
2. Prompt
3. Chains
4. Memory
5. RAG

Slide : https://www.canva.com/design/DAGP0vJmKVA/H9YpTkyQTQhg1r6-eEbfdA/edit?utm_content=DAGP0vJmKVA&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton

Reference : https://github.com/svnnynior/introduction-to-langchain-workshop

####**Install dependencies**

In [ ]:
!pip install langchain==0.0.349

In [ ]:
!pip install pypdf

####**Call API (OpenAI) - gpt-3.5-turbo-instruct & gpt-3.5-turbo**
All OpenAI model overview: https://platform.openai.com/docs/models/overview

***Note You have to pay for the OpenAI API to use it.

In [ ]:
!pip install openai==1.3.9

In [ ]:
!pip install llama-index==0.5.6

In [ ]:
!pip install python-dotenv

In [ ]:
import os
import sys
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("api.env")) # read local .env file

#use.env file
#openai.organization = os.environ['ORGANIZATION']
#openai.api_key  = os.environ['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

In [ ]:
#or use your API key string
#os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0)
print(llm("hello"))




Hello! How are you doing today?


In [ ]:
from langchain.schema import (AIMessage,HumanMessage,SystemMessage)
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(model_name= "gpt-3.5-turbo",temperature=0.3)
message= [SystemMessage(content="You are normal machine"),HumanMessage(content="hello")]
response=chat(message)

In [ ]:
print(response.content,end="\n")

Hello! How can I assist you today?


####**Prompt**


In [ ]:
chat = ChatOpenAI(model_name= "gpt-3.5-turbo",temperature=0.3)
message= [SystemMessage(content="You are genius ev car seller"),HumanMessage(content="Suggest me an ev car if i have 30000 dollars.")]
response=chat(message)

In [ ]:
print(response.content,end="\n")

If you have a budget of $30,000 for an electric vehicle, I would recommend considering the Nissan Leaf. The Nissan Leaf is a popular and reliable electric car that offers good range, comfortable interior, and a smooth driving experience. With your budget, you should be able to find a well-equipped used or new Nissan Leaf that fits your needs. Be sure to check out different trim levels and options to find the best fit for you.


In [ ]:
from prompt_toolkit.formatted_text.base import Template
from langchain import PromptTemplate

template = "You are genius ev car seller. Suggest me two {concept} if i have 30000 dollars"

prompt = PromptTemplate(template=template,input_variables=["concept"])

In [ ]:
print(prompt.format(concept="ev car"))

You are genius ev car seller. Suggest me two ev car if i have 30000 dollars


In [ ]:
llm(prompt.format(concept="ev car"))

'\n\n1. Tesla Model 3 - Starting at $35,000, the Tesla Model 3 is a popular and highly-rated electric car. It has a range of up to 263 miles per charge and can go from 0-60 mph in just 5.3 seconds.\n\n2. Chevrolet Bolt EV - Starting at $36,620, the Chevrolet Bolt EV has a range of up to 259 miles per charge and can go from 0-60 mph in 6.5 seconds. It also has a spacious interior and advanced technology features.'

####**Chains**
Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

LCEL is great for constructing your own chains, but it’s also nice to have chains that you can use off-the-shelf. There are two types of off-the-shelf chains that LangChain supports:

Chains that are built with LCEL. In this case, LangChain offers a higher-level constructor method. However, all that is being done under the hood is constructing a chain with LCEL.

[Legacy] Chains constructed by subclassing from a legacy Chain class. These chains do not use LCEL under the hood but are rather standalone classes.

In [ ]:
#Simple Sequential Chain
from langchain.chains import LLMChain
from prompt_toolkit.formatted_text.base import Template
from langchain import PromptTemplate
template = "what is the best name to descirbe a store that sale {product}." #First Template
prompt = PromptTemplate(template=template,input_variables=["product"]) #First Prompt Template
chain_one = LLMChain(llm=llm,prompt=prompt) #First chain

In [ ]:
template_2 = "Please describe {store_name} by writing 20 words." #Second Template
prompt_2 = PromptTemplate(template=template_2,input_variables=["store_name"]) #Second Prompt Template
chain_two = LLMChain(llm=llm,prompt=prompt_2) #Second chain

In [ ]:
from langchain.chains import SimpleSequentialChain
overall = SimpleSequentialChain(chains=[chain_one,chain_two],verbose=True)
explanation = overall.run("ev car")
print(explanation)



> Entering new SimpleSequentialChain chain...


"Electric Avenue Auto Emporium"


"Electric Avenue Auto Emporium is a car dealership that specializes in electric and hybrid vehicles, offering a sustainable and modern driving experience."

> Finished chain.


"Electric Avenue Auto Emporium is a car dealership that specializes in electric and hybrid vehicles, offering a sustainable and modern driving experience."


####**Memory**
Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

We call this ability to store information about past interactions "memory". LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or incorporated seamlessly into a chain.

Most of memory-related functionality in LangChain is marked as beta. This is for two reasons:

Most functionality (with some exceptions, see below) are not production ready

Most functionality (with some exceptions, see below) work with Legacy chains, not the newer LCEL syntax.

The main exception to this is the ChatMessageHistory functionality. This functionality is largely production ready and does integrate with LCEL.

LCEL Runnables: For an overview of how to use ChatMessageHistory with LCEL runnables, see these docs

Integrations: For an introduction to the various ChatMessageHistory integrations, see these docs

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(model_name= "gpt-3.5-turbo",temperature=0.3)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm,memory=memory,verbose=True)

In [ ]:
#without memory
conversation.predict(input="tell me secret word")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: tell me secret word
AI:

> Finished chain.


"I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?"

In [ ]:
#add secret word to memory
conversation.predict(input="please remember secret word is computer")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: tell me secret word
AI: I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?
Human: please remember secret word is computer
AI:

> Finished chain.


'Thank you for sharing that with me. I will remember that the secret word is "computer." Is there anything else you would like me to remember or assist you with?'

In [ ]:
conversation.predict(input="hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: tell me secret word
AI: I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?
Human: please remember secret word is computer
AI: Thank you for sharing that with me. I will remember that the secret word is "computer." Is there anything else you would like me to remember or assist you with?
Human: hello
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [ ]:
#with memory
conversation.predict(input="tell me secret word")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: tell me secret word
AI: I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?
Human: please remember secret word is computer
AI: Thank you for sharing that with me. I will remember that the secret word is "computer." Is there anything else you would like me to remember or assist you with?
Human: hello
AI: Hello! How can I assist you today?
Human: tell me secret word
AI:

> Finished chain.


'The secret word is "computer." Thank you for reminding me. Is there anything else you would like to know or discuss?'

In [ ]:
print(memory.buffer)

Human: tell me secret word
AI: I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?
Human: please remember secret word is computer
AI: Thank you for sharing that with me. I will remember that the secret word is "computer." Is there anything else you would like me to remember or assist you with?
Human: hello
AI: Hello! How can I assist you today?
Human: tell me secret word
AI: The secret word is "computer." Thank you for reminding me. Is there anything else you would like to know or discuss?


**Add input and output to memory**

In [ ]:
memory.save_context({"input":"Do you like me?"},{"output":"I dont like you."})

In [ ]:
print(memory.buffer)

Human: tell me secret word
AI: I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?
Human: please remember secret word is computer
AI: Thank you for sharing that with me. I will remember that the secret word is "computer." Is there anything else you would like me to remember or assist you with?
Human: hello
AI: Hello! How can I assist you today?
Human: tell me secret word
AI: The secret word is "computer." Thank you for reminding me. Is there anything else you would like to know or discuss?
Human: Do you like me?
AI: I dont like you.


**ConversationBufferWindowMemory**

ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
llm = ChatOpenAI(model_name= "gpt-3.5-turbo",temperature=0.3)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm,memory=memory,verbose=True)

In [ ]:
conversation.predict(input="please remember secret word is computer")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: please remember secret word is computer
AI:

> Finished chain.


'Got it! I will remember that the secret word is "computer." Is there anything else you would like me to remember or help you with?'

In [ ]:
conversation.predict(input="hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: please remember secret word is computer
AI: Got it! I will remember that the secret word is "computer." Is there anything else you would like me to remember or help you with?
Human: hello
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [ ]:
conversation.predict(input="tell me secret word")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hello
AI: Hello! How can I assist you today?
Human: tell me secret word
AI:

> Finished chain.


"I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?"

**ConversationTokenBufferMemory**

ConversationTokenBufferMemory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

In [ ]:
llm = ChatOpenAI(temperature=0)
memory = ConversationTokenBufferMemory(llm=llm,max_token_limit=50)
conversation = ConversationChain(llm=llm,memory=memory,verbose=True)

In [ ]:
conversation.predict(input="please remember secret word is computer")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: please remember secret word is computer
AI:

> Finished chain.


'Got it! I\'ll remember that the secret word is "computer." Is there anything else you\'d like me to remember or help you with?'

In [ ]:
conversation.predict(input="hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: please remember secret word is computer
AI: Got it! I'll remember that the secret word is "computer." Is there anything else you'd like me to remember or help you with?
Human: hello
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [ ]:
conversation.predict(input="tell me secret word")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hello
AI: Hello! How can I assist you today?
Human: tell me secret word
AI:

> Finished chain.


"I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?"

In [ ]:
memory.load_memory_variables({})

{'history': "Human: tell me secret word\nAI: I'm sorry, I cannot provide you with a secret word as I do not have access to that information. Is there anything else I can help you with?"}

In [ ]:
llm = ChatOpenAI(temperature=0)
memory = ConversationTokenBufferMemory(llm=llm,max_token_limit=500)
conversation = ConversationChain(llm=llm,memory=memory,verbose=True)

In [ ]:
conversation.predict(input="please remember secret word is computer")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: please remember secret word is computer
AI:

> Finished chain.


'Got it! The secret word is "computer." Is there anything else you would like me to remember for you?'

In [ ]:
conversation.predict(input="hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: please remember secret word is computer
AI: Got it! The secret word is "computer." Is there anything else you would like me to remember for you?
Human: hello
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [ ]:
conversation.predict(input="tell me secret word")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: please remember secret word is computer
AI: Got it! The secret word is "computer." Is there anything else you would like me to remember for you?
Human: hello
AI: Hello! How can I assist you today?
Human: tell me secret word
AI:

> Finished chain.


'The secret word you asked me to remember is "computer." Is there anything else you would like to know or do?'

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: please remember secret word is computer\nAI: Got it! The secret word is "computer." Is there anything else you would like me to remember for you?\nHuman: hello\nAI: Hello! How can I assist you today?\nHuman: tell me secret word\nAI: The secret word you asked me to remember is "computer." Is there anything else you would like to know or do?'}

**ConversationSummaryMemory**

This type of memory creates a summary of the conversation over time. This can be useful for condensing information from the conversation over time. Conversation summary memory summarizes the conversation as it happens and stores the current summary in memory. This memory can then be used to inject the summary of the conversation so far into a prompt/chain. This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/Toyota Research Institute Unveils New Generative AI Technique for Vehicle Design - Toyota USA Newsroom.pdf")

In [ ]:
story = loader.load()

In [ ]:
len(story) # จํานวนหน้า

7

In [ ]:
[len(story[e].page_content) for e in range (len(story)) ] #จํานวนคําในเเต่ละหน้า

[1015, 1475, 1033, 1063, 913, 418, 403]

In [ ]:
sum([len(story[e].page_content) for e in range (len(story)) ]) #จํานวนคําทั้งหมด

6320

In [ ]:
story[0].page_content #อยากดูขนาด text

'HOME > INNOVATION\xa0>\xa0RESEARCH & DEVELOPMENT\nNew Technique Gives Designers Added Capabilities by Incorporating\nEngineering Constraints Into Generative AI Models\nLOS ALTOS, Calif. (June 20, 2023) – Toyota Research Institute (TRI) today unveiled a\ngenerative artiﬁcial intelligence (AI) technique to amplify vehicle designers. Currently,\ndesigners can leverage publicly available text-to-image generative AI tools as an early\nstep in their creative process. With TRI’s new technique, designers can add initial design\nsketches and engineering constraints into this process, cutting down the iterations\nneeded to reconcile design and engineering considerations.\nRESEA RCH  &  D EVELO PM EN T\nToyota Research Institute Unveils New Generative AI\nTechnique for Vehicle Des ign\nJune 20, 20238/31/24, 1:20 PM Toyota Research Institute Unveils New Generative AI Technique for Vehicle Design - Toyota USA Newsroom\nhttps://pressroom.toyota.com/toyota-research-institute-unveils-new-generative-a

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm,max_token_limit=500)

In [ ]:
memory.save_context({"input":"what is today news ?"},{"output":f"{story}"})

In [ ]:
memory.load_memory_variables({})

{'history': "System: The human asks for today's news, and the AI provides information about Toyota Research Institute unveiling a new generative AI technique for vehicle design. This technique incorporates engineering constraints into the design process, allowing designers to optimize performance metrics while maintaining stylistic prompts. Toyota aims to improve aerodynamics and design electrified vehicles more efficiently with this new tool."}

In [ ]:
len(memory.load_memory_variables({})["history"]) #จํานวนคําหลังสรุุป

434

In [ ]:
Conversation = ConversationChain(llm=llm,memory=memory,verbose=True)

In [ ]:
Conversation.predict(input="why toyota develop Generative AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks for today's news, and the AI provides information about Toyota Research Institute unveiling a new generative AI technique for vehicle design. This technique incorporates engineering constraints into the design process, allowing designers to optimize performance metrics while maintaining stylistic prompts. Toyota aims to improve aerodynamics and design electrified vehicles more efficiently with this new tool.
Human: why toyota develop Generative AI?
AI:

> Finished chain.


'Toyota developed Generative AI to improve the efficiency of designing electrified vehicles. By incorporating engineering constraints into the design process, designers can optimize performance metrics while maintaining stylistic prompts. This allows Toyota to enhance aerodynamics and create more efficient vehicle designs.'

In [ ]:
Conversation.predict(input="what type of Generative AI that Toyota developing?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks for today's news, and the AI provides information about Toyota Research Institute unveiling a new generative AI technique for vehicle design. This technique incorporates engineering constraints into the design process, allowing designers to optimize performance metrics while maintaining stylistic prompts. Toyota aims to improve aerodynamics and design electrified vehicles more efficiently with this new tool.
Human: why toyota develop Generative AI?
AI: Toyota developed Generative AI to improve the efficiency of designing electrified vehicles. By incorporating engineering constraints into the design process, designers can optimize performance metr

'Toyota is developing a new generative AI technique for vehicle design. This technique incorporates engineering constraints into the design process, allowing designers to optimize performance metrics while maintaining stylistic prompts. It is specifically designed to improve aerodynamics and design electrified vehicles more efficiently.'

In [ ]:
memory.load_memory_variables({})

{'history': "System: The human asks for today's news, and the AI provides information about Toyota Research Institute unveiling a new generative AI technique for vehicle design. This technique incorporates engineering constraints into the design process, allowing designers to optimize performance metrics while maintaining stylistic prompts. Toyota aims to improve aerodynamics and design electrified vehicles more efficiently with this new tool.\nHuman: why toyota develop Generative AI?\nAI: Toyota developed Generative AI to improve the efficiency of designing electrified vehicles. By incorporating engineering constraints into the design process, designers can optimize performance metrics while maintaining stylistic prompts. This allows Toyota to enhance aerodynamics and create more efficient vehicle designs.\nHuman: what type of Generative AI that Toyota developing?\nAI: Toyota is developing a new generative AI technique for vehicle design. This technique incorporates engineering constr

####**RAG**
RAG is a technique for augmenting LLM knowledge with additional data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

LangChain has a number of components designed to help build Q&A applications, and RAG applications more generally.

In [ ]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [ ]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

**Text Embedding**

In [ ]:
!pip install sentence_transformers==2.2.2 faiss-cpu==1.7.4

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


**Using Retriver**

In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

template = """Answer the question based only on the following context:

{context}

Question: {question}

Let's think step by step.

Answer:
"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} # RunablePassthrough make a question consistency.
    | prompt
    | llm

)

In [ ]:
import langchain

langchain.debug = True #show all step
result = chain.invoke("What does the speech say about Russia?")
langchain.debug = False

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 5:chain:format_docs] Entering Chain run with input:
[inputs]
[chain

In [ ]:
print(result.content)

The speech describes Russia, specifically Vladimir Putin, as attempting to shake the foundations of the free world through his actions in Ukraine. It mentions that Putin thought he could make the world bend to his ways, but instead, he met a wall of strength from the Ukrainian people. The speech highlights the economic sanctions and actions being taken against Russia, such as releasing oil reserves and cutting off access to technology and financial systems. It also mentions that Putin's actions have isolated Russia from the world and weakened the country economically and militarily. The speech emphasizes that democracies are rising to the moment and choosing peace and security over autocracy.
